In [3]:
import tensorflow as tf
import numpy as np
import keras
import time
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.datasets import cifar10
from callbacks import all_callbacks

%matplotlib inline
seed = 0
np.random.seed(seed)

In [4]:
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.cifar10.load_data()

x_train = x_train.astype('float32') / 255
x_test = x_test.astype('float32') / 255

y_train = tf.keras.utils.to_categorical(y_train, 10)
y_test = tf.keras.utils.to_categorical(y_test, 10)

cifar10_classes=['airplane','automobile','bird','cat','deer','dog','frog','horse','ship','truck']

In [5]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Input, MaxPooling2D, Flatten, Activation
from qkeras import QActivation, QDense, QConv2D
from qkeras.quantizers import quantized_bits, quantized_relu
from tensorflow.keras.regularizers import l1

In [ ]:
model = Sequential()

model.add(
    Input(shape=(32, 32, 3))
)

# Conv 1
model.add(
    QConv2D(
    96,
    kernel_size=(5, 5),
    strides=(1, 1),
    padding='valid',
    name='conv1',
    kernel_quantizer=quantized_bits(8, 4, alpha=1),
    bias_quantizer=quantized_bits(8, 4, alpha=1),
    kernel_initializer='lecun_uniform',
    kernel_regularizer=l1(0.0001)
    )
)
model.add(
    QActivation(activation=quantized_relu(4), name='relu1')
)
model.add(
    MaxPooling2D(pool_size=(3, 3), strides=(2, 2), padding='valid', name='pool1')
)

# Conv 2
model.add(
    QConv2D(
    256,
    kernel_size=(5, 5),
    strides=(1, 1),
    padding='valid',
    name='conv2',
    kernel_quantizer=quantized_bits(8, 4, alpha=1),
    bias_quantizer=quantized_bits(8, 4, alpha=1),
    kernel_initializer='lecun_uniform',
    kernel_regularizer=l1(0.0001)
    )
)
model.add(
    QActivation(activation=quantized_relu(4), name='relu2')
)
model.add(
    MaxPooling2D(pool_size=(3, 3), strides=(2, 2), padding='valid', name='pool2')
)

# Conv 3
model.add(
    QConv2D(
    384,
    kernel_size=(3, 3),
    strides=(1, 1),
    padding='same',
    name='conv3',
    kernel_quantizer=quantized_bits(8, 4, alpha=1),
    bias_quantizer=quantized_bits(8, 4, alpha=1),
    kernel_initializer='lecun_uniform',
    kernel_regularizer=l1(0.0001)
    )
)
model.add(
    QActivation(activation=quantized_relu(4), name='relu3')
)

# Conv 4
model.add(
    QConv2D(
    384,
    kernel_size=(3, 3),
    strides=(1, 1),
    padding='same',
    name='conv4',
    kernel_quantizer=quantized_bits(8, 4, alpha=1),
    bias_quantizer=quantized_bits(8, 4, alpha=1),
    kernel_initializer='lecun_uniform',
    kernel_regularizer=l1(0.0001)
    )
)
model.add(
    QActivation(activation=quantized_relu(4), name='relu4')
)

# Conv 5
model.add(
    QConv2D(
    256,
    kernel_size=(3, 3),
    strides=(1, 1),
    padding='same',
    name='conv5',
    kernel_quantizer=quantized_bits(8, 4, alpha=1),
    bias_quantizer=quantized_bits(8, 4, alpha=1),
    kernel_initializer='lecun_uniform',
    kernel_regularizer=l1(0.0001)
    )
)
model.add(
    QActivation(activation=quantized_relu(4), name='relu5')
)
model.add(
    MaxPooling2D(pool_size=(3, 3), strides=(2, 2), padding='valid', name='pool5')
)

# Flatten
model.add(Flatten())

# FC 1
model.add(
    QDense(
        4096,
        name='fc1',
        kernel_quantizer=quantized_bits(8, 4, alpha=1),
        bias_quantizer=quantized_bits(8, 4, alpha=1),
        kernel_initializer='lecun_uniform',
        kernel_regularizer=l1(0.0001)
    )
)
model.add(
    QActivation(activation=quantized_relu(4), name='relu6')
)

# FC 2
model.add(
    QDense(
        4096,
        name='fc2',
        kernel_quantizer=quantized_bits(8, 4, alpha=1),
        bias_quantizer=quantized_bits(8, 4, alpha=1),
        kernel_initializer='lecun_uniform',
        kernel_regularizer=l1(0.0001)
    )
)
model.add(
    QActivation(activation=quantized_relu(4), name='relu7')
)

# FC 3
model.add(
    QDense(
        10,
        name='output',
        kernel_quantizer=quantized_bits(8, 4, alpha=1),
        bias_quantizer=quantized_bits(8, 4, alpha=1),
        kernel_initializer='lecun_uniform',
        kernel_regularizer=l1(0.0001)
    )
)

model.add(Activation(activation='softmax', name='softmax'))

model.summary()

In [ ]:
from tensorflow_model_optimization.python.core.sparsity.keras import prune, pruning_callbacks, pruning_schedule
from tensorflow_model_optimization.sparsity.keras import strip_pruning

pruning_params = {"pruning_schedule": pruning_schedule.ConstantSparsity(0.75, begin_step=2000, frequency=100)}
model = prune.prune_low_magnitude(model, **pruning_params)

In [ ]:
train = True
if train:
    adam = Adam(learning_rate=0.001)
    model.compile(
        optimizer=adam, 
        loss=['categorical_crossentropy'], 
        metrics=['accuracy'])
    
    model.fit(
        x_train,
        y_train,
        batch_size=4096,
        epochs=500,
        validation_data=(x_test, y_test),
        shuffle=True,
    )
    
    model = strip_pruning(model)
    model.save('AlexNet_Weights/AlexNet_check_best_model.h5')
else:
    from tensorflow.keras.models import load_model
    from qkeras.utils import _add_supported_quantized_objects

    co = {}
    _add_supported_quantized_objects(co)
    model = load_model('AlexNet_Weights/AlexNet_check_best_model.h5', custom_objects=co)

Epoch 1/500
13/13 [==============================] - 28s 2s/step - loss: 10.0603 - accuracy: 0.1059 - val_loss: 9.9746 - val_accuracy: 0.1117
Epoch 2/500
13/13 [==============================] - 22s 2s/step - loss: 9.9090 - accuracy: 0.1125 - val_loss: 9.8203 - val_accuracy: 0.1257
Epoch 3/500
13/13 [==============================] - 22s 2s/step - loss: 9.7556 - accuracy: 0.1310 - val_loss: 9.6686 - val_accuracy: 0.1444
Epoch 4/500
13/13 [==============================] - 23s 2s/step - loss: 9.6078 - accuracy: 0.1517 - val_loss: 9.5284 - val_accuracy: 0.1570
Epoch 5/500
13/13 [==============================] - 23s 2s/step - loss: 9.4614 - accuracy: 0.1580 - val_loss: 9.3675 - val_accuracy: 0.1633
Epoch 6/500
13/13 [==============================] - 23s 2s/step - loss: 9.2919 - accuracy: 0.1674 - val_loss: 9.2095 - val_accuracy: 0.1723
Epoch 7/500
13/13 [==============================] - 23s 2s/step - loss: 9.1459 - accuracy: 0.1675 - val_loss: 9.0492 - val_accuracy: 0.1696
Epoch 8/500



***callbacks***
saving losses to AlexNet_Weights/Qkeras/losses.log

Epoch 00184: val_loss did not improve from 1.25419

Epoch 00184: val_loss did not improve from 1.25419

Epoch 00184: saving model to AlexNet_Weights/Qkeras/KERAS_check_model_last.h5

Epoch 00184: saving model to AlexNet_Weights/Qkeras/KERAS_check_model_last_weights.h5

***callbacks end***

Epoch 185/5000
49/49 [==============================] - 1s 19ms/step - loss: 1.2055 - accuracy: 0.5815 - val_loss: 1.2653 - val_accuracy: 0.5578

***callbacks***
saving losses to AlexNet_Weights/Qkeras/losses.log

Epoch 00185: val_loss did not improve from 1.25419

Epoch 00185: val_loss did not improve from 1.25419

Epoch 00185: saving model to AlexNet_Weights/Qkeras/KERAS_check_model_last.h5

Epoch 00185: saving model to AlexNet_Weights/Qkeras/KERAS_check_model_last_weights.h5

***callbacks end***

Epoch 186/5000
49/49 [==============================] - 1s 19ms/step - loss: 1.1997 - accuracy: 0.5870 - val_loss: 1.2821 - val_accurac

49/49 [==============================] - 1s 19ms/step - loss: 1.2078 - accuracy: 0.5811 - val_loss: 1.2601 - val_accuracy: 0.5619

***callbacks***
saving losses to AlexNet_Weights/Qkeras/losses.log

Epoch 00200: val_loss did not improve from 1.25419

Epoch 00200: val_loss did not improve from 1.25419

Epoch 00200: saving model to AlexNet_Weights/Qkeras/KERAS_check_model_last.h5

Epoch 00200: saving model to AlexNet_Weights/Qkeras/KERAS_check_model_last_weights.h5

Epoch 00200: saving model to AlexNet_Weights/Qkeras/KERAS_check_model_epoch200.h5

***callbacks end***

Epoch 201/5000
49/49 [==============================] - 1s 19ms/step - loss: 1.1974 - accuracy: 0.5854 - val_loss: 1.2611 - val_accuracy: 0.5596

***callbacks***
saving losses to AlexNet_Weights/Qkeras/losses.log

Epoch 00201: val_loss did not improve from 1.25419

Epoch 00201: val_loss did not improve from 1.25419

Epoch 00201: saving model to AlexNet_Weights/Qkeras/KERAS_check_model_last.h5

Epoch 00201: saving model to A


Epoch 00215: saving model to AlexNet_Weights/Qkeras/KERAS_check_model_last_weights.h5

***callbacks end***

Epoch 216/5000
49/49 [==============================] - 1s 19ms/step - loss: 1.1989 - accuracy: 0.5846 - val_loss: 1.2806 - val_accuracy: 0.5552

***callbacks***
saving losses to AlexNet_Weights/Qkeras/losses.log

Epoch 00216: val_loss did not improve from 1.25419

Epoch 00216: val_loss did not improve from 1.25419

Epoch 00216: saving model to AlexNet_Weights/Qkeras/KERAS_check_model_last.h5

Epoch 00216: saving model to AlexNet_Weights/Qkeras/KERAS_check_model_last_weights.h5

***callbacks end***

Epoch 217/5000
49/49 [==============================] - 1s 19ms/step - loss: 1.1972 - accuracy: 0.5858 - val_loss: 1.2842 - val_accuracy: 0.5544

***callbacks***
saving losses to AlexNet_Weights/Qkeras/losses.log

Epoch 00217: val_loss did not improve from 1.25419

Epoch 00217: val_loss did not improve from 1.25419

Epoch 00217: saving model to AlexNet_Weights/Qkeras/KERAS_check_mode


Epoch 00231: saving model to AlexNet_Weights/Qkeras/KERAS_check_model_last_weights.h5

***callbacks end***

Epoch 232/5000
49/49 [==============================] - 1s 19ms/step - loss: 1.2053 - accuracy: 0.5815 - val_loss: 1.2525 - val_accuracy: 0.5636

***callbacks***
saving losses to AlexNet_Weights/Qkeras/losses.log

Epoch 00232: val_loss improved from 1.25419 to 1.25249, saving model to AlexNet_Weights/Qkeras/KERAS_check_best_model.h5

Epoch 00232: val_loss improved from 1.25419 to 1.25249, saving model to AlexNet_Weights/Qkeras/KERAS_check_best_model_weights.h5

Epoch 00232: saving model to AlexNet_Weights/Qkeras/KERAS_check_model_last.h5

Epoch 00232: saving model to AlexNet_Weights/Qkeras/KERAS_check_model_last_weights.h5

***callbacks end***

Epoch 233/5000
49/49 [==============================] - 1s 19ms/step - loss: 1.1947 - accuracy: 0.5857 - val_loss: 1.2790 - val_accuracy: 0.5571

***callbacks***
saving losses to AlexNet_Weights/Qkeras/losses.log

Epoch 00233: val_loss di

...

49/49 [==============================] - 1s 19ms/step - loss: 1.1865 - accuracy: 0.5906 - val_loss: 1.2702 - val_accuracy: 0.5590

***callbacks***
saving losses to AlexNet_Weights/Qkeras/losses.log

Epoch 00440: val_loss did not improve from 1.24756

Epoch 00440: val_loss did not improve from 1.24756

Epoch 00440: saving model to AlexNet_Weights/Qkeras/KERAS_check_model_last.h5

Epoch 00440: saving model to AlexNet_Weights/Qkeras/KERAS_check_model_last_weights.h5

Epoch 00440: saving model to AlexNet_Weights/Qkeras/KERAS_check_model_epoch440.h5

***callbacks end***

Epoch 441/5000
49/49 [==============================] - 1s 19ms/step - loss: 1.2021 - accuracy: 0.5832 - val_loss: 1.2719 - val_accuracy: 0.5552

***callbacks***
saving losses to AlexNet_Weights/Qkeras/losses.log

Epoch 00441: val_loss did not improve from 1.24756

Epoch 00441: val_loss did not improve from 1.24756

Epoch 00441: saving model to AlexNet_Weights/Qkeras/KERAS_check_model_last.h5

Epoch 00441: saving model to A


***callbacks end***

Epoch 456/5000
49/49 [==============================] - 1s 19ms/step - loss: 1.2028 - accuracy: 0.5859 - val_loss: 1.2973 - val_accuracy: 0.5492

***callbacks***
saving losses to AlexNet_Weights/Qkeras/losses.log

Epoch 00456: val_loss did not improve from 1.24756

Epoch 00456: val_loss did not improve from 1.24756

Epoch 00456: saving model to AlexNet_Weights/Qkeras/KERAS_check_model_last.h5

Epoch 00456: saving model to AlexNet_Weights/Qkeras/KERAS_check_model_last_weights.h5

***callbacks end***

Epoch 457/5000
49/49 [==============================] - 1s 19ms/step - loss: 1.1968 - accuracy: 0.5867 - val_loss: 1.2571 - val_accuracy: 0.5620

***callbacks***
saving losses to AlexNet_Weights/Qkeras/losses.log

Epoch 00457: val_loss did not improve from 1.24756

Epoch 00457: val_loss did not improve from 1.24756

Epoch 00457: saving model to AlexNet_Weights/Qkeras/KERAS_check_model_last.h5

Epoch 00457: saving model to AlexNet_Weights/Qkeras/KERAS_check_model_last_w


***callbacks end***

Epoch 472/5000
49/49 [==============================] - 1s 19ms/step - loss: 1.1821 - accuracy: 0.5937 - val_loss: 1.2782 - val_accuracy: 0.5547

***callbacks***
saving losses to AlexNet_Weights/Qkeras/losses.log

Epoch 00472: val_loss did not improve from 1.24756

Epoch 00472: val_loss did not improve from 1.24756

Epoch 00472: saving model to AlexNet_Weights/Qkeras/KERAS_check_model_last.h5

Epoch 00472: saving model to AlexNet_Weights/Qkeras/KERAS_check_model_last_weights.h5

***callbacks end***

Epoch 473/5000
49/49 [==============================] - 1s 19ms/step - loss: 1.1840 - accuracy: 0.5918 - val_loss: 1.2867 - val_accuracy: 0.5524

***callbacks***
saving losses to AlexNet_Weights/Qkeras/losses.log

Epoch 00473: val_loss did not improve from 1.24756

Epoch 00473: val_loss did not improve from 1.24756

Epoch 00473: saving model to AlexNet_Weights/Qkeras/KERAS_check_model_last.h5

Epoch 00473: saving model to AlexNet_Weights/Qkeras/KERAS_check_model_last_w

49/49 [==============================] - 1s 19ms/step - loss: 1.1838 - accuracy: 0.5919 - val_loss: 1.2540 - val_accuracy: 0.5625

***callbacks***
saving losses to AlexNet_Weights/Qkeras/losses.log

Epoch 00488: val_loss did not improve from 1.24756

Epoch 00488: val_loss did not improve from 1.24756

Epoch 00488: saving model to AlexNet_Weights/Qkeras/KERAS_check_model_last.h5

Epoch 00488: saving model to AlexNet_Weights/Qkeras/KERAS_check_model_last_weights.h5

***callbacks end***

Epoch 489/5000
49/49 [==============================] - 1s 19ms/step - loss: 1.1911 - accuracy: 0.5907 - val_loss: 1.2702 - val_accuracy: 0.5588

***callbacks***
saving losses to AlexNet_Weights/Qkeras/losses.log

Epoch 00489: val_loss did not improve from 1.24756

Epoch 00489: val_loss did not improve from 1.24756

Epoch 00489: saving model to AlexNet_Weights/Qkeras/KERAS_check_model_last.h5

Epoch 00489: saving model to AlexNet_Weights/Qkeras/KERAS_check_model_last_weights.h5

***callbacks end***

Epoch


***callbacks end***

Epoch 504/5000
49/49 [==============================] - 1s 19ms/step - loss: 1.1937 - accuracy: 0.5890 - val_loss: 1.2611 - val_accuracy: 0.5609

***callbacks***
saving losses to AlexNet_Weights/Qkeras/losses.log

Epoch 00504: val_loss did not improve from 1.24756

Epoch 00504: val_loss did not improve from 1.24756

Epoch 00504: saving model to AlexNet_Weights/Qkeras/KERAS_check_model_last.h5

Epoch 00504: saving model to AlexNet_Weights/Qkeras/KERAS_check_model_last_weights.h5

***callbacks end***

Epoch 505/5000
49/49 [==============================] - 1s 19ms/step - loss: 1.1883 - accuracy: 0.5891 - val_loss: 1.2524 - val_accuracy: 0.5650

***callbacks***
saving losses to AlexNet_Weights/Qkeras/losses.log

Epoch 00505: val_loss did not improve from 1.24756

Epoch 00505: val_loss did not improve from 1.24756

Epoch 00505: saving model to AlexNet_Weights/Qkeras/KERAS_check_model_last.h5

Epoch 00505: saving model to AlexNet_Weights/Qkeras/KERAS_check_model_last_w

In [6]:
predictions = model.predict(x_test)
predictions

array([[2.66918577e-02, 2.03259345e-02, 9.37569886e-02, ...,
        1.77372526e-02, 7.25205764e-02, 1.06017459e-02],
       [1.31269932e-01, 2.38280177e-01, 5.72271459e-03, ...,
        1.26530752e-02, 2.96399713e-01, 2.76679873e-01],
       [2.13042498e-01, 2.26893321e-01, 1.17750624e-02, ...,
        1.10562472e-02, 4.32464182e-01, 6.34381846e-02],
       ...,
       [9.54090524e-03, 2.64941365e-03, 1.35425314e-01, ...,
        2.36636639e-01, 1.29627471e-03, 6.26319135e-03],
       [1.09956056e-01, 3.82661581e-01, 1.23265259e-01, ...,
        7.32107610e-02, 3.03804269e-03, 3.25347036e-02],
       [2.92310165e-03, 5.80477249e-03, 2.67454190e-03, ...,
        8.89720142e-01, 2.02545983e-04, 8.20204522e-03]], dtype=float32)